In [2]:
import warnings
import tensorflow as tf
import numpy as np
import pandas as pd
import json
from modules.CARLA import carla

seed = 44

tf.random.set_seed(seed)
np.random.seed(seed)

Appending path to carla module D:\Research\ecemosp\src\modules\CARLA
Appending path to carla module D:\Research\ecemosp\src\modules\CARLA


C:\Users\ignacy\AppData\Roaming\Python\Python37\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[INFO] Using Python-MIP package version 1.12.0 [model.py <module>]


In [3]:
from carla.data.catalog import CsvCatalog
import json

with open('../data/adult_constraints.json', 'r') as f:
    constraints = json.load(f)

continuous = constraints['continuous']
categorical = constraints['categorical']
immutable = constraints['immutable']
columns_order = constraints['features_order']

dataset = CsvCatalog(file_path="../data/adult_prep.csv",
                    continuous=continuous,
                    categorical=categorical,
                    immutables=immutable,
                    target='income')
dataset.df.to_csv("../data/adult_prep_test.csv",index=False)

X_train = dataset.df_train
Y_train = dataset.df_train
X_test = dataset.df_test
Y_test = dataset.df_test
X_train

,age,education.num,capital.gain,capital.loss,hours.per.week,...,workclass_Private,workclass_Self-emp-inc,workclass_Self-emp-not-inc,workclass_State-gov,workclass_Without-pay
9739,0.397260,0.533333,0.000000,0.0,0.397959,...,1.0,0.0,0.0,0.0,0.0
23394,0.342466,0.800000,0.000000,0.0,0.500000,...,1.0,0.0,0.0,0.0,0.0
25744,0.287671,0.666667,0.000000,0.0,0.397959,...,1.0,0.0,0.0,0.0,0.0
2631,0.397260,0.800000,0.076881,0.0,0.448980,...,1.0,0.0,0.0,0.0,0.0
15550,0.246575,0.600000,0.000000,0.0,0.459184,...,1.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
16955,0.452055,0.533333,0.000000,0.0,0.397959,...,1.0,0.0,0.0,0.0,0.0
25773,0.547945,0.866667,0.000000,0.0,0.551020,...,1.0,0.0,0.0,0.0,0.0
27377,0.000000,0.400000,0.000000,0.0,0.112245,...,1.0,0.0,0.0,0.0,0.0
3491,0.643836,0.400000,0.036740,0.0,0.346939,...,0.0,0.0,0.0,0.0,0.0


In [7]:

X_train = X_train[columns_order].to_numpy()
Y_train = Y_train['income']
Y_train = pd.get_dummies(Y_train).to_numpy()
Y_train

array([[1, 0],
       [0, 1],
       [1, 0],
       ...,
       [1, 0],
       [1, 0],
       [1, 0]], dtype=uint8)

In [8]:
X_test = X_test[columns_order].to_numpy()
Y_test = Y_test['income']
Y_test = pd.get_dummies(Y_test).to_numpy()
Y_test

array([[1, 0],
       [1, 0],
       [1, 0],
       ...,
       [1, 0],
       [1, 0],
       [0, 1]], dtype=uint8)

In [5]:
constraints['features_count']

90

In [6]:
from tensorflow import keras


model = tf.keras.Sequential()
model.add(tf.keras.layers.Input((constraints['features_count'],)))
model.add(tf.keras.layers.Dense(64, activation='relu'))
model.add(tf.keras.layers.Dropout(0.2))
model.add(tf.keras.layers.Dense(32, activation='relu'))
model.add(tf.keras.layers.Dropout(0.2))
model.add(tf.keras.layers.Dense(16, activation='relu'))
model.add(tf.keras.layers.Dropout(0.2))
model.add(tf.keras.layers.Dense(2, activation='softmax'))


model.compile(
    optimizer='adam',
    loss=tf.keras.losses.CategoricalCrossentropy(from_logits=False),
    metrics=['accuracy']
)

In [7]:
model.fit(
    X_train, 
    Y_train,
    epochs=50,
    batch_size=128,
    validation_data=(X_test, Y_test),
    shuffle=True,
    callbacks=[
        tf.keras.callbacks.EarlyStopping(patience=10)
    ]
)

Train on 24420 samples, validate on 8141 samples
Epoch 1/50
24420/24420 [==============================] - 1s 23us/sample - loss: 0.4389 - accuracy: 0.7885 - val_loss: 0.3711 - val_accuracy: 0.8288
Epoch 2/50
 4352/24420 [====>.........................] - ETA: 0s - loss: 0.3958 - accuracy: 0.8116

c:\Users\ignacy\.conda\envs\carla_test\lib\site-packages\keras\engine\training_v1.py:2057: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates = self.state_updates


24420/24420 [==============================] - 0s 14us/sample - loss: 0.3773 - accuracy: 0.8238 - val_loss: 0.3524 - val_accuracy: 0.8397
Epoch 3/50
24420/24420 [==============================] - 0s 16us/sample - loss: 0.3614 - accuracy: 0.8326 - val_loss: 0.3418 - val_accuracy: 0.8409
Epoch 4/50
24420/24420 [==============================] - 0s 13us/sample - loss: 0.3524 - accuracy: 0.8359 - val_loss: 0.3348 - val_accuracy: 0.8454
Epoch 5/50
24420/24420 [==============================] - 0s 16us/sample - loss: 0.3420 - accuracy: 0.8414 - val_loss: 0.3335 - val_accuracy: 0.8461
Epoch 6/50
24420/24420 [==============================] - 0s 16us/sample - loss: 0.3367 - accuracy: 0.8415 - val_loss: 0.3276 - val_accuracy: 0.8449
Epoch 7/50
24420/24420 [==============================] - 0s 15us/sample - loss: 0.3310 - accuracy: 0.8474 - val_loss: 0.3250 - val_accuracy: 0.8468
Epoch 8/50
24420/24420 [==============================] - 0s 17us/sample - loss: 0.3262 - accuracy: 0.8473 - val_loss

In [8]:
model.save('../models/adult_NN.h5', overwrite=True, save_format='h5')

In [11]:
model.save('../models/adult_NN', overwrite=True)
model_test = tf.keras.models.load_model('../models/adult_NN.h5')
model_test.predict(X_test[0:2], verbose=1)

c:\Users\ignacy\.conda\envs\carla_test\lib\site-packages\keras\engine\training_v1.py:2079: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,


array([[8.3331722e-01, 1.6668279e-01],
       [9.9963045e-01, 3.6948692e-04]], dtype=float32)

In [12]:
model.load_weights('../models/adult_NN.h5')

In [13]:
X_test[0].shape

(90,)

In [14]:
model.predict(X_test[0:2], verbose=1)


c:\Users\ignacy\.conda\envs\carla_test\lib\site-packages\keras\engine\training_v1.py:2079: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,


array([[8.3331722e-01, 1.6668279e-01],
       [9.9963045e-01, 3.6948692e-04]], dtype=float32)

RandomForest

In [5]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from joblib import dump, load

In [9]:
classifier = RandomForestClassifier(random_state=seed, criterion='entropy')
classifier.fit(X_train, Y_train)

RandomForestClassifier(criterion='entropy', random_state=44)

In [10]:
classifier.score(X_test, Y_test)

0.8502640953199853

In [11]:
dump(classifier, '../models/adult_RF.joblib')

['../models/adult_RF.joblib']

In [ ]:
clf = load('../models/adult_RF.joblib')

In [ ]:
print(y_test[0:5])
clf.predict(X_test[0:5])